Problem Statement: <br/>
    
Given a non-empty string s and a dictionary wordDict containing a list of non-empty words, add spaces in s to construct a sentence where each word is a valid dictionary word. Return all such possible sentences. <br/>

Note: <br/>
    The same word in the dictionary may be reused multiple times in the segmentation. <br/>
    You may assume the dictionary does not contain duplicate words. <br/>

Example 1: <br/>
Input: <br/>
s = "catsanddog" <br/>
wordDict = ["cat", "cats", "and", "sand", "dog"] <br/>
Output: <br/>
[ <br/>
  "cats and dog", <br/>
  "cat sand dog" <br/>
] <br/>

Example 2: <br/>
Input: <br/>
s = "pineapplepenapple" <br/>
wordDict = ["apple", "pen", "applepen", "pine", "pineapple"] <br/>
Output: <br/>
[ <br/>
  "pine apple pen apple", <br/>
  "pineapple pen apple", <br/>
  "pine applepen apple" <br/>
] <br/>
Explanation: Note that you are allowed to reuse a dictionary word. <br/>

Example 3: <br/>
Input: <br/>
s = "catsandog" <br/>
wordDict = ["cats", "dog", "sand", "and", "cat"] <br/>
Output: []

# Top Down Recursion - O(2 ^ N) runtime, O(N) space

In [1]:
from typing import List

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        wordSet = set(wordDict)
        n = len(s)
        result = []
        self.wordBreakRecursive(s, wordSet, 0, 0, [], result)
        return result
    
    def wordBreakRecursive(self, s, wordSet, i, j, currList, result):
        if j == len(s):
            if i == j: 
                result.append(' '.join(currList))
                return True
            return False

        self.wordBreakRecursive(s, wordSet, i, j+1, currList, result)
        
        new = False
        curr = s[i:j+1]
        if curr in wordSet:
            currList.append(curr)
            self.wordBreakRecursive(s, wordSet, j+1, j+1, currList, result)
            currList.pop()

# Top Down DP - O(N ^ 2 + 2 ^ N + W) runtime, O(2 ^ N * N )  space

In [2]:
from typing import List

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        if set(Counter(s).keys()) > set(Counter("".join(wordDict)).keys()):
            return []
        
        self.results = []
        self.wordBreakRecursive(s, wordDict, 0, 0, [])
        
        return self.results
        
    def wordBreakRecursive(self, s: str, wordDict: List[str], startIndex: int, currIndex: int, result: List[int]):
        n = len(s)
        if startIndex == n and currIndex == n:
            i = 0
            words = []
            for j in result:
                words.append(s[i: j + 1])
                i = j + 1
            self.results.append(' '.join(words))
            return
        
        elif currIndex == n:
            return

        if s[startIndex: currIndex + 1] in wordDict:
            new_result = result.copy()
            new_result.append(currIndex)
            self.wordBreakRecursive(s, wordDict, currIndex + 1, currIndex + 1, new_result)
        self.wordBreakRecursive(s, wordDict, startIndex, currIndex + 1, result)

# Top Down DP with memoization - O(N ^ 2 + 2 ^ N + W) runtime, O(2 ^ N * N + W) space, where N is the length of the string and W is the number of words in the dictionary

In [ ]:
from typing import List

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        wordSet = set(wordDict)
        # table to map a string to its corresponding words break
        # {string: [['word1', 'word2'...], ['word3', 'word4', ...]]}
        memo = defaultdict(list)

        #@lru_cache(maxsize=None)    # alternative memoization solution
        def _wordBreak_topdown(s):
            """ return list of word lists """
            if not s:
                return [[]]  # list of empty list

            if s in memo:
                # returned the cached solution directly.
                return memo[s]

            for endIndex in range(1, len(s)+1):
                word = s[:endIndex]
                if word in wordSet:
                    # move forwards to break the postfix into words
                    for subsentence in _wordBreak_topdown(s[endIndex:]):
                        memo[s].append([word] + subsentence)
            return memo[s]

        # break the input string into lists of words list
        _wordBreak_topdown(s)

        # chain up the lists of words into sentences.
        return [" ".join(words) for words in memo[s]]

# Bottom Up DP - O(N ^ 2 + 2 ^ N + W) runtime, O(2 ^ N * N + N ^ 2 + W) space, where N is the length of the string and W is the number of words in the dictionary

In [6]:
from typing import List
from collections import Counter

class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> List[str]:
        # quick check on the characters,
        #   otherwise it would exceed the time limit for certain test cases.
        if set(Counter(s).keys()) > set(Counter("".join(wordDict)).keys()):
            return []

        wordSet = set(wordDict)

        dp = [[]] * (len(s)+1)
        dp[0] = [[0]]

        for endIndex in range(1, len(s)+1):
            stops = []
            for startIndex in range(0, endIndex):
                word = s[startIndex:endIndex]
                if word in wordSet:
                    stops.append([startIndex, endIndex])
            dp[endIndex] = stops
        
        ret = []
        def wordDFS(sentence, dp_index):
            if dp_index == 0:
                ret.append(" ".join(sentence))
                return

            for start, end in dp[dp_index]:
                word = s[start:end]
                wordDFS([word] + sentence, start)

        wordDFS([], len(s))

        return ret

In [7]:
instance = Solution()
instance.wordBreak("pineapplepenapple", ["apple", "pen", "applepen", "pine", "pineapple"])

['pine applepen apple', 'pineapple pen apple', 'pine apple pen apple']